# Proyecto: Selección de Regiones para Nuevos Pozos Petrolíferos en OilyGiant

# Introducción

OilyGiant es una empresa de extracción de petróleo en búsqueda de las mejores ubicaciones para abrir 200 nuevos pozos petrolíferos. Para seleccionar las mejores regiones, utilizaremos un modelo de regresión lineal para predecir el volumen de reservas de petróleo. Además, analizaremos los riesgos y ganancias potenciales mediante la técnica de bootstrapping. El objetivo es encontrar la región que maximice las ganancias y minimice los riesgos de pérdida.

Este proyecto incluye el análisis de datos de tres regiones diferentes. Utilizaremos la información sobre características geológicas y volúmenes de reservas para desarrollar nuestro modelo.

# Objetivo

El objetivo de este proyecto es:
- Entrenar un modelo de regresión lineal para predecir el volumen de reservas de petróleo en tres regiones.
- Seleccionar los mejores 200 pozos petrolíferos por región basándonos en las predicciones del modelo.
- Analizar las ganancias y riesgos asociados a cada región utilizando bootstrapping y recomendar la mejor opción para la empresa.

# Índice

1. Carga y Exploración de los Datos
2. Preparación de los Datos
3. Entrenamiento del Modelo
4. Evaluación del Modelo
5. Cálculo de Ganancias
6. Análisis de Riesgos y Bootstrapping
7. Conclusiones

##  Carga y Exploración de los Datos

En esta sección, cargaremos los datos de las tres regiones y exploraremos su estructura para entender qué tipo de información contienen. Analizaremos las primeras filas y verificaremos la información general de cada conjunto de datos.

In [3]:
# Importamos las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
# Cargamos los datos de las tres regiones

# Código original
'''
geo_data_0 = pd.read_csv('geo_data_0.csv')
geo_data_1 = pd.read_csv('geo_data_1.csv')
geo_data_2 = pd.read_csv('geo_data_2.csv')
'''

# Código del revisor
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')



In [5]:
# Exploramos las primeras filas de cada dataset
print("Datos de la Región 0")
print(geo_data_0.head(), "\n")

print("Datos de la Región 1")
print(geo_data_1.head(), "\n")

print("Datos de la Región 2")
print(geo_data_2.head(), "\n")

Datos de la Región 0
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

Datos de la Región 1
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305 

Datos de la Región 2
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746 



### Información General de los Datos

A continuación, revisaremos la información general de cada conjunto de datos para verificar el tipo de variables y la cantidad de valores faltantes o inconsistentes que podamos encontrar.

In [6]:
# Información general de los datasets
print("Información de la Región 0")
print(geo_data_0.info(), "\n")

print("Información de la Región 1")
print(geo_data_1.info(), "\n")

print("Información de la Región 2")
print(geo_data_2.info(), "\n")

Información de la Región 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

Información de la Región 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

Información de la Región 2
<class 'pandas.core.frame.DataFrame'>

##  Preparación de los Datos

En esta sección, dividiremos los datos en conjuntos de entrenamiento y validación para cada región en una proporción de 75:25. Esto nos permitirá entrenar un modelo de regresión lineal y evaluar su rendimiento en el conjunto de validación.

In [7]:
# División de los datos en entrenamiento y validación (75% para entrenamiento y 25% para validación)
geo_data_0_train, geo_data_0_valid = train_test_split(geo_data_0, test_size=0.25, random_state=42)
geo_data_1_train, geo_data_1_valid = train_test_split(geo_data_1, test_size=0.25, random_state=42)
geo_data_2_train, geo_data_2_valid = train_test_split(geo_data_2, test_size=0.25, random_state=42)

# Verificamos el tamaño de los conjuntos divididos
print("Tamaño de los datos de la Región 0 - Entrenamiento:", geo_data_0_train.shape)
print("Tamaño de los datos de la Región 0 - Validación:", geo_data_0_valid.shape, "\n")

print("Tamaño de los datos de la Región 1 - Entrenamiento:", geo_data_1_train.shape)
print("Tamaño de los datos de la Región 1 - Validación:", geo_data_1_valid.shape, "\n")

print("Tamaño de los datos de la Región 2 - Entrenamiento:", geo_data_2_train.shape)
print("Tamaño de los datos de la Región 2 - Validación:", geo_data_2_valid.shape, "\n")

Tamaño de los datos de la Región 0 - Entrenamiento: (75000, 5)
Tamaño de los datos de la Región 0 - Validación: (25000, 5) 

Tamaño de los datos de la Región 1 - Entrenamiento: (75000, 5)
Tamaño de los datos de la Región 1 - Validación: (25000, 5) 

Tamaño de los datos de la Región 2 - Entrenamiento: (75000, 5)
Tamaño de los datos de la Región 2 - Validación: (25000, 5) 



##  Entrenamiento del Modelo

En esta sección, entrenaremos un modelo de regresión lineal para predecir el volumen de reservas de petróleo. Utilizaremos los datos de entrenamiento de cada región y evaluaremos el modelo con el conjunto de validación para medir su precisión.

In [8]:
# Entrenamiento de un modelo de regresión lineal para la Región 0
X_train_0 = geo_data_0_train[['f0', 'f1', 'f2']]
y_train_0 = geo_data_0_train['product']
X_valid_0 = geo_data_0_valid[['f0', 'f1', 'f2']]
y_valid_0 = geo_data_0_valid['product']

model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)

# Predicciones para el conjunto de validación
predictions_0 = model_0.predict(X_valid_0)

# Cálculo del RMSE
rmse_0 = mean_squared_error(y_valid_0, predictions_0, squared=False)
mean_reserves_0 = predictions_0.mean()

print(f"Región 0 - Volumen promedio de reservas predicho: {mean_reserves_0:.2f}")
print(f"Región 0 - RMSE del modelo: {rmse_0:.2f}\n")

# Repetimos el proceso para las Regiones 1 y 2
# Región 1
X_train_1 = geo_data_1_train[['f0', 'f1', 'f2']]
y_train_1 = geo_data_1_train['product']
X_valid_1 = geo_data_1_valid[['f0', 'f1', 'f2']]
y_valid_1 = geo_data_1_valid['product']

model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)

predictions_1 = model_1.predict(X_valid_1)
rmse_1 = mean_squared_error(y_valid_1, predictions_1, squared=False)
mean_reserves_1 = predictions_1.mean()

print(f"Región 1 - Volumen promedio de reservas predicho: {mean_reserves_1:.2f}")
print(f"Región 1 - RMSE del modelo: {rmse_1:.2f}\n")

# Región 2
X_train_2 = geo_data_2_train[['f0', 'f1', 'f2']]
y_train_2 = geo_data_2_train['product']
X_valid_2 = geo_data_2_valid[['f0', 'f1', 'f2']]
y_valid_2 = geo_data_2_valid['product']

model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)

predictions_2 = model_2.predict(X_valid_2)
rmse_2 = mean_squared_error(y_valid_2, predictions_2, squared=False)
mean_reserves_2 = predictions_2.mean()

print(f"Región 2 - Volumen promedio de reservas predicho: {mean_reserves_2:.2f}")
print(f"Región 2 - RMSE del modelo: {rmse_2:.2f}\n")

Región 0 - Volumen promedio de reservas predicho: 92.40
Región 0 - RMSE del modelo: 37.76

Región 1 - Volumen promedio de reservas predicho: 68.71
Región 1 - RMSE del modelo: 0.89

Región 2 - Volumen promedio de reservas predicho: 94.77
Región 2 - RMSE del modelo: 40.15



## Cálculo de Ganancias

En esta sección, seleccionaremos los 200 pozos con los valores de predicción más altos en cada región. Utilizaremos estos pozos para calcular las ganancias esperadas, teniendo en cuenta las condiciones del negocio.

In [9]:
# Definimos una función para calcular la ganancia
def calcular_ganancia(predicciones):
    # Seleccionamos las 200 mejores predicciones
    mejores_pozos = predicciones.sort_values(ascending=False)[:200]
    # Calculamos las ganancias
    ganancia = mejores_pozos.sum() * 4500  # 4500 dólares por cada unidad de reservas
    return ganancia

# Cálculo de ganancias para la Región 0
ganancia_0 = calcular_ganancia(pd.Series(predictions_0))
print(f"Ganancia estimada para la Región 0: ${ganancia_0:.2f}")

# Cálculo de ganancias para la Región 1
ganancia_1 = calcular_ganancia(pd.Series(predictions_1))
print(f"Ganancia estimada para la Región 1: ${ganancia_1:.2f}")

# Cálculo de ganancias para la Región 2
ganancia_2 = calcular_ganancia(pd.Series(predictions_2))
print(f"Ganancia estimada para la Región 2: ${ganancia_2:.2f}")

Ganancia estimada para la Región 0: $138966584.80
Ganancia estimada para la Región 1: $124869381.15
Ganancia estimada para la Región 2: $133779815.14


## Análisis de Riesgos y Bootstrapping

En esta sección, utilizaremos la técnica de bootstrapping para estimar la distribución de ganancias de cada región. Calcularemos el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas (probabilidad de ganancia negativa).

In [10]:
def bootstrap_riesgos(predicciones, n_simulaciones=1000):
    valores = []
    for _ in range(n_simulaciones):
        muestra = predicciones.sample(n=500, replace=True)  # Tomamos una muestra aleatoria de 500 pozos
        ganancia_muestra = calcular_ganancia(muestra)  # Calculamos la ganancia
        valores.append(ganancia_muestra)
    
    valores = pd.Series(valores)
    
    # Calculamos el intervalo de confianza del 95%
    intervalo_confianza = np.percentile(valores, [2.5, 97.5])
    
    # Calculamos el riesgo de pérdida (porcentaje de muestras con ganancias negativas)
    riesgo_perdida = (valores < 100_000_000).mean() * 100  # 100 millones es el presupuesto
    
    return valores.mean(), intervalo_confianza, riesgo_perdida

# Realizamos el bootstrapping para cada región
media_0, intervalo_0, riesgo_0 = bootstrap_riesgos(pd.Series(predictions_0))
media_1, intervalo_1, riesgo_1 = bootstrap_riesgos(pd.Series(predictions_1))
media_2, intervalo_2, riesgo_2 = bootstrap_riesgos(pd.Series(predictions_2))

# Mostramos los resultados
print(f"Región 0 - Beneficio promedio: ${media_0:.2f}, Intervalo de confianza 95%: {intervalo_0}, Riesgo de pérdida: {riesgo_0:.2f}%")
print(f"Región 1 - Beneficio promedio: ${media_1:.2f}, Intervalo de confianza 95%: {intervalo_1}, Riesgo de pérdida: {riesgo_1:.2f}%")
print(f"Región 2 - Beneficio promedio: ${media_2:.2f}, Intervalo de confianza 95%: {intervalo_2}, Riesgo de pérdida: {riesgo_2:.2f}%")

Región 0 - Beneficio promedio: $103292216.93, Intervalo de confianza 95%: [1.01212345e+08 1.05519084e+08], Riesgo de pérdida: 0.20%
Región 1 - Beneficio promedio: $104302078.62, Intervalo de confianza 95%: [1.00291532e+08 1.08355816e+08], Riesgo de pérdida: 2.20%
Región 2 - Beneficio promedio: $102693154.55, Intervalo de confianza 95%: [1.00851644e+08 1.04500787e+08], Riesgo de pérdida: 0.30%


##  Conclusiones

Después de analizar los datos de las tres regiones, entrenar los modelos de regresión lineal y evaluar los beneficios y riesgos utilizando la técnica de bootstrapping, llegamos a las siguientes conclusiones:

1. **Región 0**:
   - Beneficio promedio estimado: $103,356,406.49
   - Intervalo de confianza del 95%: [101,148,855, 105,567,245]
   - Riesgo de pérdida: 0.10%
   
2. **Región 1**:
   - Beneficio promedio estimado: $104,394,216.11
   - Intervalo de confianza del 95%: [100,397,664, 108,160,966]
   - Riesgo de pérdida: 1.80%

3. **Región 2**:
   - Beneficio promedio estimado: $102,744,337.63
   - Intervalo de confianza del 95%: [100,823,208, 104,720,806]
   - Riesgo de pérdida: 0.10%

### Recomendación Final

- La **Región 0** presenta un buen equilibrio entre beneficio y bajo riesgo. Con un riesgo de pérdida del 0.10% y un beneficio promedio alto, es una opción sólida para el desarrollo de pozos petrolíferos.
- La **Región 1**, aunque ofrece un mayor beneficio promedio, conlleva un mayor riesgo (1.80% de pérdida). Si la empresa está dispuesta a asumir más riesgos, podría considerar esta región.
- La **Región 2** también es una opción conservadora con un bajo riesgo de pérdida (0.10%) y un beneficio promedio similar al de la Región 0.

En resumen, mi recomendación es desarrollar los nuevos pozos en la **Región 0** debido a su bajo riesgo y buen rendimiento promedio.